In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from vardata import years, team_abbr, columns, teams_idx
from results_processor import *

In [2]:
# read in data csv to df
df = pd.read_csv('stats/final_data.csv', index_col=[0])

In [3]:
# preview the data to ensure it was read correctly
print('Dimension of dataset= ', df.shape)
df.head() # To get first n rows from the dataset default value of n is 5

Dimension of dataset=  (3359, 75)


,home_team,away_team,ht_points_for,ht_yards,ht_plays,ht_yards_per_play,ht_turnovers,ht_fumbles_lost,ht_first_downs,ht_completions,...,at_4D_conversions,at_4D_conversion_percentage,at_red_zone_attempts,at_red_zone_tds,at_red_zone_td_percentage,ht_score,at_score,total_points,ht_pts_diff,ht_vegas_line
0,nor,min,366.0,5481.0,1022.0,5.35,28.5,10.0,313.5,391.5,...,11.5,56.15,45.5,20.0,43.95,14,9,23,5,-5.0
1,pit,atl,331.5,5422.0,975.0,5.60,24.5,9.0,289.5,331.0,...,10.5,67.90,49.0,25.5,50.15,15,9,24,6,1.5
2,buf,mia,308.0,4913.5,971.0,5.05,29.0,13.0,263.5,292.0,...,5.0,50.00,44.0,25.5,57.00,10,15,25,-5,3.0
3,chi,det,351.5,5064.0,970.5,5.20,30.0,12.5,291.5,303.0,...,8.0,54.35,43.5,24.5,56.00,19,14,33,5,-6.5
4,nwe,cin,456.5,5566.0,971.5,5.75,18.0,7.5,312.5,328.5,...,12.5,64.20,52.5,27.0,51.55,38,24,62,14,-5.5


In [4]:
X = df.values[:, 2:70]  # get input values from first two columns
y = df.values[:, 73]  # get output values from last coulmn
m = len(y) # number of training examples

In [5]:
# preview the data
print('Total no of samples (m) = %s \n' %(m))

# Show only first 5 records
for i in range(5):
    print('X =', X[i, ], ', y =', y[i])

Total no of samples (m) = 3359 

X = [366.0 5481.0 1022.0 5.35 28.5 10.0 313.5 391.5 595.0 3904.0 29.0 18.5
 6.25 207.0 398.5 1577.0 9.5 3.95 90.5 94.0 835.0 16.0 179.0 33.65 14.6
 294.0 4969.5 965.0 5.15 31.0 13.5 281.0 305.5 499.5 3100.0 13.5 17.5
 5.800000000000001 154.5 431.0 1869.5 14.5 4.35 99.0 94.0 735.0 27.5 181.0
 28.35 15.85 215.0 96.5 44.8 10.5 5.0 47.75 53.5 28.5 53.65 204.0 70.5
 34.55 20.0 11.5 56.150000000000006 45.5 20.0 43.95] , y = 5
X = [331.5 5422.0 975.0 5.6 24.5 9.0 289.5 331.0 519.5 3613.0 22.5 15.5 6.5
 179.0 418.5 1809.0 12.0 4.35 96.5 108.0 953.0 14.0 179.5 31.5 12.8 323.0
 4943.5 1035.5 4.75 26.0 11.0 312.5 362.0 585.0 3496.0 21.5 15.0 5.6 191.0
 415.0 1447.5 9.5 3.4 86.0 72.0 657.0 35.5 178.5 32.5 13.15 209.5 86.5
 41.2 11.0 5.5 46.45 46.0 23.5 51.4 224.5 91.0 40.1 15.5 10.5 67.9 49.0
 25.5 50.15] , y = 6
X = [308.0 4913.5 971.0 5.05 29.0 13.0 263.5 292.0 510.5 3253.0 23.0 16.0 5.95
 166.0 424.0 1660.5 7.0 3.95 83.0 83.5 681.5 14.5 181.5 28.4 14.55 349.0
 5

In [6]:
# split data into train and test sets
# 10 years of data for training, 3 years of data for testing
X_train = X[:2560]
X_test = X[2560:]
y_train = y[:2560]
y_test = y[2560:]

In [7]:
# create a function for different types of transformations
def basis_functions(X):
    return np.concatenate([X, np.sin(X), np.cos(X)], axis=1)

# Create a FunctionTransformer object to apply the custom function to the data
basis_transformer = preprocessing.FunctionTransformer(basis_functions)

In [13]:
# define scaler and transform input data
scaler = preprocessing.MinMaxScaler().fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

In [95]:
# Create a pipeline that scales the data, performs polynomial expansion and custom basis function expansion, and then fits a linear regression model
model_r = Pipeline([
    ('poly', preprocessing.PolynomialFeatures(degree=2)),
    ('basis', basis_transformer),
    ('linear', linear_model.Ridge())
])

In [96]:
# create linear regression model using basis function expansion
model_r.fit(X_scaled_train,y_train)

Pipeline(steps=[('poly', PolynomialFeatures()),
                ('basis',
                 FunctionTransformer(func=<function basis_functions at 0x15cd84790>)),
                ('linear', Ridge())])

In [97]:
# get results of model on test data
results = np.array([])
for each in X_scaled_test:
    outcome = model_r.predict([each])
    results = np.append(results, [outcome])

In [98]:
for i in range(5):
    print(results[i])

19.51964315366015
1.8333614421935405
34.25150797377025
-7.412280041831878
-10.429305451969807


In [99]:
results_df = pd.DataFrame(columns = ['ht_pts_diff', 'ht_vegas_line', 'model_pred'])
results_df['ht_pts_diff'] = df['ht_pts_diff'][2560:]
results_df['ht_vegas_line'] = df['ht_vegas_line'][2560:]
results_df = results_df.reset_index(drop = True)
results_df['model_pred'] = results.tolist()

In [100]:
results_df.to_csv('results/lin_reg_bfe_results.csv')

In [101]:
# get results of model on training data
train_results = np.array([])
for each in X_scaled_train:
    outcome = model_r.predict([each])
    train_results = np.append(train_results, [outcome])

In [102]:
train_results_df = pd.DataFrame(columns = ['ht_pts_diff', 'ht_vegas_line', 'model_pred'])
train_results_df['ht_pts_diff'] = df['ht_pts_diff'][:2560]
train_results_df['ht_vegas_line'] = df['ht_vegas_line'][:2560]
train_results_df = train_results_df.reset_index(drop = True)
train_results_df['model_pred'] = train_results.tolist()

*************************************************************
**The following are the results of the model on Training Data:**
*************************************************************

In [103]:
process_results(train_results_df)

Total Number of Games:
 2560
Winning Bets Count:
 1701
No Bet Placed Count:
 95
Winning Percentage:
 0.6901
______________________________________
Winner Correctly Predicted Count:
 1909
Accuracy in Classifaction:
 0.7457
______________________________________
Mean Squared Error of Results:
 112.9186
Root Mean Squared Error of Results:
 10.6263


*************************************************************
**The following are the results of the model on Testing Data:**
*************************************************************

In [104]:
process_results(results_df)

Total Number of Games:
 799
Winning Bets Count:
 463
No Bet Placed Count:
 17
Winning Percentage:
 0.5921
______________________________________
Winner Correctly Predicted Count:
 521
Accuracy in Classifaction:
 0.6521
______________________________________
Mean Squared Error of Results:
 177.2092
Root Mean Squared Error of Results:
 13.3120
